# Particle Swarm Optimization (PSO) Project Report

## Introduction

Particle Swarm Optimization (PSO) is a computational method used for optimizing a problem by iteratively improving a candidate solution with regard to a given measure of quality. This project explores three different implementations of the PSO algorithm and compares their performance.

## Rastrigin Function

The Rastrigin function is a non-convex function used as a performance test problem for optimization algorithms. It is highly multimodal, meaning it has a large number of local minima. The function is defined by:

$$f(x) = An + \sum_{i=1}^{n} \left[ x_i^2 - A \cos(2\pi x_i) \right]$$


where $A = 10$, $x_i ∈ [-5.12, 5.12]$

Global Minimum : $f(x) = 0$ for $x = 0$

Maximum Function Value: $xi ∈ [±4.52299366...,...,±4.52299366...]$

#### code:

```python

def rastrigin(X):
    if isinstance(X[0],(int,float)):
        X = [[X[i]] for i in range(len(X))]
    
    val = []
    for xi in X:
        fx = 10 * len(xi) + sum(np.array(xi) ** 2 - 10 * np.cos(2 * np.pi * np.array(xi)))
        val.append(fx)
    return np.array(val)

```

### Population Generation

The `generate_particle` function is responsible for initializing the swarm (population of particles) and their velocities.

```python
def generate_particle(num_variables, swarm_size, x_min, x_max):
    swarm = np.random.uniform(x_min, x_max, (swarm_size, num_variables))
    velocity = np.zeros_like(swarm)
    return swarm, velocity
```

#### Explanation

The `generate_particle` function initializes the swarm and their velocities as follows:

1. **Swarm Initialization**:
    - The swarm is a 2D array where each row represents a particle and each column represents a variable (dimension) of the problem.
    - `np.random.uniform(x_min, x_max, (swarm_size, num_variables))` generates a swarm of particles with random positions. Each position is uniformly distributed between `x_min` and `x_max`.
    - `swarm_size` specifies the number of particles in the swarm.
    - `num_variables` specifies the number of dimensions (variables) for each particle.

2. **Velocity Initialization**:
    - The velocity is a 2D array of the same shape as the swarm.
    - `np.zeros_like(swarm)` initializes the velocity of each particle to zero. This means that initially, particles have no movement.

3. **Return Values**:
    - The function returns the initialized `swarm` and `velocity`.

## PSO Algorithms

### 1. **PSO Algorithm 1**

This is the first implementation of the PSO algorithm. It uses the following parameters:

- **Number of Iterations:** 100
- **Swarm Size:** 50
- **Number of Variables:** 5
- **Search Space:** [-5.12, 5.12]
- **Inertia Weight (W):** 0.9
- **Cognitive Coefficient (C1):** 0.2
- **Social Coefficient (C2):** 0.7
- **Velocity Coefficient (E):** 0.2

#### code:

```python
def pso(num_iterations,swarm_size,num_variables,x_min,x_max,W,C1,C2,E):
    swarm,velocity = generate_particle(num_variables,swarm_size,x_min,x_max)
    local_best = np.copy(swarm)
    global_best = np.full_like(local_best,local_best[index_finder(swarm)])
    
    for z in range(num_iterations):
        swarm2 = swarm + (E * velocity)
        swarm2 = np.clip(swarm2,x_min,x_max)
        
        ras_swarm = np.array(rastrigin(swarm))
        ras_swarm2 = np.array(rastrigin(swarm2))
        
        for i in range(swarm_size):
            if ras_swarm2[i] < ras_swarm[i]:
                local_best[i] = swarm2[i]
                continue
            if min(ras_swarm2) < min(ras_swarm):
                global_best = np.full_like(local_best,local_best[index_finder(swarm2)])
                # print(global_best[0])
                continue
            
        velocity = ((W*(W-.4)*velocity*z)/num_iterations) + C1 * np.random.uniform(0,1) * (local_best - swarm) + C2 * np.random.uniform(0,1) * (global_best - swarm)
    
    return global_best[0]
```

### 2. **PSO Algorithm 2**

This is the second implementation of the PSO algorithm. It uses the following parameters:

- **Number of Iterations:** 100
- **Swarm Size:** 50
- **Number of Variables:** 5
- **Search Space:** [-5.12, 5.12]
- **Inertia Weight (α):** 0.9  (controls the influence of the particle's previous velocity on its current velocity)
- **Cognitive Coefficient (β):** 0.2 (controls the influence of the particle's own best-known position on its current velocity)
- **Social Coefficient (γ):** 0.7 (controls the influence of the swarm's best-known position on the particle's current velocity)
- **Velocity Coefficient (ε):** 0.2 (introduces stochasticity into the particle's movement)

#### code:

```python
def pso1(num_iterations, swarm_size, num_variables, x_min, x_max, alpha, beta, gamma, epsilon):
    
    swarm, velocity = generate_particle(num_variables, swarm_size, x_min, x_max)
    
    for i in range(num_iterations):
        
        swarm2 = swarm + epsilon * velocity
        swarm2 = np.clip(swarm2,x_min,x_max)
        
        dicx = {tuple(swarm[i]): rastrigin(swarm)[i] for i in range(swarm_size)}
        dicy = {tuple(swarm2[i]): rastrigin(swarm2)[i] for i in range(swarm_size)}

        if i == 0:
            
            local_best = swarm
            global_best = list(min(dicx, key=lambda k: dicx[k]))
        else:
            for j in range(swarm_size):
                key1, value1 = list(dicx.items())[j]
                key2, value2 = list(dicy.items())[j]
                
                local_best[j] = list(key2) if value2 < value1 else list(key1)
                
                global_best = list(min(dicy, key=lambda k: dicy[k])) if np.argmin(dicy) < np.argmin(dicx) else list(min(dicx, key=lambda k: dicx[k]))
                      

        velocity = (alpha * velocity + np.random.uniform(0, beta) * (np.array(local_best) - np.array(swarm)) + np.random.uniform(0, gamma) * (np.full_like(swarm, global_best) - np.array(swarm)))
        
        
    return global_best
```

### 3. **PSO Algorithm 3**

This is the third implementation of the PSO algorithm. It uses the following parameters:

- **Number of Iterations:** 100
- **Swarm Size:** 50
- **Number of Variables:** 5
- **Search Space:** [-5.12, 5.12]
- **Inertia Weight (α):** 0.9  (controls the influence of the particle's previous velocity on its current velocity)
- **Cognitive Coefficient (β):** 0.2 (controls the influence of the particle's own best-known position on its current velocity)
- **Social Coefficient (γ):** 0.7 (controls the influence of the swarm's best-known position on the particle's current velocity)
- **Velocity Coefficient (ε):** 0.2 (introduces stochasticity into the particle's movement)

#### code:

```python
def pso2(num_iterations, swarm_size, num_variables, x_min, x_max, alpha, beta, gamma, epsilon):
    def rastrigin(X):
        return 10 * len(X) + sum([(x ** 2 - 10 * np.cos(2 * np.pi * x)) for x in X])

    def generate_particle(num_variables, swarm_size, x_min, x_max):
        swarm = np.random.uniform(x_min, x_max, (swarm_size, num_variables))
        velocity = np.zeros_like(swarm)
        return swarm, velocity

    swarm, velocity = generate_particle(num_variables, swarm_size, x_min, x_max)
    swarm = np.clip(swarm, x_min, x_max)
    swarm_positions = [swarm.tolist()]

    for i in range(num_iterations):

        swarm2 = swarm + epsilon * velocity
        swarm2 = np.clip(swarm2, x_min, x_max)
        
        dicx = {tuple(swarm[i]): rastrigin(swarm[i]) for i in range(swarm_size)}
        dicy = {tuple(swarm2[i]): rastrigin(swarm2[i]) for i in range(swarm_size)}

        if i == 0:
            local_best = swarm
            global_best = list(min(dicx, key=dicx.get))
        else:
            for j in range(swarm_size):
                key1, value1 = list(dicx.items())[j]
                key2, value2 = list(dicy.items())[j]
                
                local_best[j] = list(key2) if value2 < value1 else list(key1)
                
                global_best = list(min(dicy, key=dicy.get)) if min(dicy.values()) < min(dicx.values()) else list(min(dicx, key=dicx.get))

        velocity = (alpha * velocity + np.random.uniform(0, beta) * (np.array(local_best) - np.array(swarm)) + np.random.uniform(0, gamma) * (np.full_like(swarm, global_best) - np.array(swarm)))
        
        swarm = swarm2
        swarm_positions.append(swarm.tolist())

    return global_best, swarm_positions
```

The global best position found by this algorithm is:

| Algorithm | Global Best Position | Rastrigin Function Value | Execution Time (s) |
|-----------|----------------------|--------------------------|--------------------|
| PSO 1     | [ 2.20412036,  3.94632764,  0.28219694, -1.9361664 ,  0.88730092] | 47.97453437 | 42.6 |
| PSO 2     | [1.0145939658519847, -0.0041799238350148475, 0.00047935063627344617, 0.0005206643559654415, 0.9818669763215216] | 2.10387461 | 2.2 |
| PSO 3     | [-3.951197759123789, 2.0690898271096465, -0.9768304627327755, -2.133493834483567, 1.7961338305104446] | 40.58288207 | 0.1 |

## Simulation Demonstration

Below is an animation demonstrating the simulation of the PSO algorithms:

![PSO Simulation](pso_animation.gif)

## Comparative Study

Complexity: All three algorithms have similar computational complexity, primarily $(O(n \cdot m \cdot k))$, where (n) is the number of iterations, (m) is the swarm size, and (k) is the number of variables.

Convergence: PSO Algorithm 2 showed the best convergence to the global minimum with the lowest Rastrigin function value. This suggests that the specific parameter tuning in Algorithm 2 was more effective for this problem.

Stability: The results indicate that Algorithm 2 is more stable and consistent in finding a better solution compared to Algorithms 1 and 3.

Exploration vs. Exploitation: The parameters in Algorithm 2 provided a better balance between exploration (searching new areas) and exploitation (refining known good areas), leading to better overall performance.

## Conclusion

In conclusion, all three PSO algorithms were able to find a global best position with similar Rastrigin function values. However, the specific positions found by each algorithm varied. This demonstrates the stochastic nature of the PSO algorithm and the importance of parameter tuning for achieving optimal results.


**Note:** The animation file is attached to provide a visual demonstration of the PSO algorithms in 2-variables.


In [49]:
import numpy as np

# Define Rastrigin Function


In [50]:
def rastrigin(X):
    if isinstance(X[0],(int,float)):
        X = [[X[i]] for i in range(len(X))]
        
    val = []
    for xi in X:
        fx = 10 * len(xi) + sum(np.array(xi) ** 2 - 10 * np.cos(2 * np.pi * np.array(xi)))
        val.append(fx)
    return np.array(val)

# Initiate population

In [51]:
def generate_particle(num_variables,swarm_size,x_min,x_max):
    swarm = np.random.uniform(x_min,x_max,(swarm_size,num_variables))

    velocity = np.zeros_like(swarm)

    return swarm , velocity

In [52]:

def index_finder(arr):
    mini = 0
    for i in range(len(arr)):
        if rastrigin(arr)[i] < rastrigin(arr)[mini]:
            mini = i
    return mini 

# Particle Swarm Optimisation

In [53]:
def pso(num_iterations,swarm_size,num_variables,x_min,x_max,W,C1,C2,E):
    swarm,velocity = generate_particle(num_variables,swarm_size,x_min,x_max)
    local_best = np.copy(swarm)
    global_best = np.full_like(local_best,local_best[index_finder(swarm)])
    
    for z in range(num_iterations):
        swarm2 = swarm + (E * velocity)
        swarm2 = np.clip(swarm2,x_min,x_max)
        
        ras_swarm = np.array(rastrigin(swarm))
        ras_swarm2 = np.array(rastrigin(swarm2))
        
        for i in range(swarm_size):
            if ras_swarm2[i] < ras_swarm[i]:
                local_best[i] = swarm2[i]
                continue
            if min(ras_swarm2) < min(ras_swarm):
                global_best = np.full_like(local_best,local_best[index_finder(swarm2)])
                # print(global_best[0])
                continue
            
        velocity = ((W*(W-.4)*velocity* z )/num_iterations) + C1 * np.random.uniform(0,1) * (local_best - swarm) + C2 * np.random.uniform(0,1) * (global_best - swarm)
    
    return global_best[0]

In [54]:
def pso1(num_iterations, swarm_size, num_variables, x_min, x_max, alpha, beta, gamma, epsilon):
    
    swarm, velocity = generate_particle(num_variables, swarm_size, x_min, x_max)
    
    for i in range(num_iterations):
        
        swarm2 = swarm + epsilon * velocity
        swarm2 = np.clip(swarm2,x_min,x_max)
        
        dicx = {tuple(swarm[i]): rastrigin(swarm)[i] for i in range(swarm_size)}
        dicy = {tuple(swarm2[i]): rastrigin(swarm2)[i] for i in range(swarm_size)}

        if i == 0:
            
            local_best = swarm
            global_best = list(min(dicx, key=lambda k: dicx[k]))
        else:
            for j in range(swarm_size):
                key1, value1 = list(dicx.items())[j]
                key2, value2 = list(dicy.items())[j]
                
                local_best[j] = list(key2) if value2 < value1 else list(key1)
                
                global_best = list(min(dicy, key=lambda k: dicy[k])) if np.argmin(dicy) < np.argmin(dicx) else list(min(dicx, key=lambda k: dicx[k]))
                # global_best = list(min(dicy, key=dicy.get)) if min(dicy.values()) < min(dicx.values()) else list(min(dicx, key=dicx.get))
                # print(global_best)      

        velocity = alpha*velocity + (np.random.uniform(0, beta) * (np.array(local_best) - np.array(swarm))) + (np.random.uniform(0, gamma) * (np.full_like(swarm, global_best) - np.array(swarm)))
        
        
    return global_best


In [55]:
def pso2(num_iterations, swarm_size, num_variables, x_min, x_max, alpha, beta, gamma, epsilon):
    def rastrigin(X):
        return 10 * len(X) + sum([(x ** 2 - 10 * np.cos(2 * np.pi * x)) for x in X])

    def generate_particle(num_variables, swarm_size, x_min, x_max):
        swarm = np.random.uniform(x_min, x_max, (swarm_size, num_variables))
        velocity = np.zeros_like(swarm)
        return swarm, velocity

    swarm, velocity = generate_particle(num_variables, swarm_size, x_min, x_max)
    swarm = np.clip(swarm, x_min, x_max)
    swarm_positions = [swarm.tolist()]

    for i in range(num_iterations):
        swarm2 = swarm + epsilon * velocity
        swarm2 = np.clip(swarm2, x_min, x_max)
        
        dicx = {tuple(swarm[i]): rastrigin(swarm[i]) for i in range(swarm_size)}    # this in pso1 is done as rastrigin(swarm)[i] maybe this works,
                                                                                    # because of the way swarms are generated, but i'm not sure
        dicy = {tuple(swarm2[i]): rastrigin(swarm2[i]) for i in range(swarm_size)}

        if i == 0:
            local_best = swarm
            global_best = list(min(dicx, key=dicx.get))
        else:
            for j in range(swarm_size):
                key1, value1 = list(dicx.items())[j]
                key2, value2 = list(dicy.items())[j]
                
                local_best[j] = list(key2) if value2 < value1 else list(key1)
                
                global_best = list(min(dicy, key=dicy.get)) if min(dicy.values()) < min(dicx.values()) else list(min(dicx, key=dicx.get))

        velocity = ((alpha*(alpha-.4)* velocity * i )/num_iterations) + (np.random.uniform(0, beta) * (np.array(local_best) - np.array(swarm))) + (np.random.uniform(0, gamma) * (np.full_like(swarm, global_best) - np.array(swarm)))
        
        swarm = swarm2
        swarm_positions.append(swarm.tolist())

    return global_best, swarm_positions

In [56]:
gb = pso(100,50,5,-5.12,+5.12,.9,.2,.7,.2)
gb

array([ 1.95085827, -0.78253951, -1.19564862, -0.78797699,  1.32251815])

In [57]:
rastrigin([gb])

array([45.34818289])

In [58]:
gb1 = pso1(100,50,5,-5.12,+5.12,.9,.2,.7,.2) # notice as the number of iterations and swarm size is increased the result is less accurate on some occasions
gb1

[0.9902503637114113,
 -0.9940551264905424,
 1.9905697613685347,
 0.006736472170628761,
 -0.0020944445098359573]

In [59]:
rastrigin([gb1])

array([5.9842628])

In [60]:
globalbest, swarm_positions = pso2(100, 50, 5, -5.12, 5.12, 0.9, 0.2, 0.7, 0.2)
print(globalbest)

[-0.04620238695089511, -2.99677185567451, 0.05504244198017408, -0.8670936841032133, -0.9479263504617537]


In [61]:
rastrigin([globalbest])

array([15.46807126])